# 1.8 - Merging the *.dta* File with the Shock Series and CPI Data

This script takes the file *Compustat_93Q1_14Q4_2.dta* created in 1.7 and adds to each observations the quarterly value for my daily data series, the quarterised Nakamura and Steinsson (2018) series, and seasonally adjusted monthly CPI data from the Federal Reserve Bank of St. Louis' *FRED* database, found at **fred.stlouisfed.org/series/CPALTT01USM661S**.

### Preamble

This script makes use of...

- Pandas
- NumPy

In [316]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import DateOffset

### Importing Quarterly Shocks and CPI into Dataframes

In [317]:
cpi_df = pd.read_csv('CPI.csv').rename(columns = {'DATE':'Date'}) # Rename date column to align with other scripts

qshocks_df = pd.read_csv('quarter_shocks.csv').rename(columns = {'Unnamed: 0':'Date'})

cpi_df['Date'] = [pd.Timestamp(date) for date in cpi_df['Date']] # Converting dates from strings to Pandas timestamps.

qshocks_df['Date'] = [pd.Timestamp(date) for date in qshocks_df['Date']]

cpi_df = cpi_df.set_index('Date') # Setting 'Date' as index

qshocks_df = qshocks_df.set_index('Date')

### Normalising Shocks

This block normalises each shock series such that a shock of 1 is an expansionary shock equivalent to the mean absolute value of shocks in the series.

In [318]:
qshocks_df['s_shock_p'] = qshocks_df['s_shock_p']/(-1*np.abs(qshocks_df['s_shock_p'].values).mean())

qshocks_df['s_shock_r'] = qshocks_df['s_shock_r']/(-1*np.abs(qshocks_df['s_shock_r'].values).mean())

qshocks_df['ns_shock_p'] = qshocks_df['ns_shock_p']/(-1*np.abs(qshocks_df['ns_shock_p'].values).mean())

qshocks_df['ns_shock_r'] = qshocks_df['ns_shock_r']/(-1*np.abs(qshocks_df['ns_shock_r'].values).mean())

### Getting Quarterly CPI Growth

In [319]:
cpi_qgrowth_df = pd.DataFrame(index = cpi_df.index[:-3], columns = ['cpi_growth'])

q = pd.DateOffset(months = 3) # Standard offset of one quarter

for date in cpi_qgrowth_df.index:
    
    growth = cpi_df.loc[date + q,'CPALTT01USM661S']/cpi_df.loc[date,'CPALTT01USM661S']
    
    cpi_qgrowth_df.loc[date,'cpi_growth'] = growth

### Importing Compustat Data

This block and those below handle a dataframe of over 400,000 rows, and as such can be expected to run slowly

In [320]:
dta_df = pd.read_stata('Compustat_93Q1_14Q4_2.dta')

### Getting "True Date" Variable in Pandas Timestamp Format

Since each firm quarter is variable (i.e. 1992 Q2 as listed could start anywhere from April 1992 to March 1993), the below code gets the true date of each quarter *start* (compustat data arrives at quarter end, but my data corresponds to quarter starts)

In [321]:
dta_df['year_quarter'] = dta_df['fyearq'] + dta_df['fqtr']/10 # Get a float format variable of yyyy.q

offsets = np.array([pd.DateOffset(months = y) for y in (10 - dta_df['fyrc'])]) # Get offset from listed date for each variable according to when fiscal year ends for firm

f_dates = np.array([pd.Timestamp(str(y)[:4] + '-' + str((int(str(y)[-1:]) - 1)*3 + 1) + '-01') for y in dta_df['year_quarter']]) # Get listed date

dta_df['true_qstart'] = f_dates + offsets

### Getting Shocks and CPI Growth Values into DataFrame

In [322]:
s_shock_p_dict = qshocks_df.s_shock_p.to_dict()

s_shock_r_dict = qshocks_df.s_shock_r.to_dict()

ns_shock_p_dict = qshocks_df.ns_shock_p.to_dict()

ns_shock_r_dict = qshocks_df.ns_shock_r.to_dict()

cpi_dict = cpi_qgrowth_df.cpi_growth.to_dict()

dta_s_shocks_p = []

dta_s_shocks_r = []

dta_ns_shocks_p = []

dta_ns_shocks_r = []

dta_cpi = []

ob = 0 # observation counter

shocks_index = qshocks_df.index

cpi_growth_index = cpi_qgrowth_df.index

dta_qstarts = dta_df['true_qstart']

for date in dta_qstarts:
    
    if date in shocks_index:
        
        dta_s_shocks_p.append(s_shock_p_dict[date])
        
        dta_s_shocks_r.append(s_shock_r_dict[date])
        
        dta_ns_shocks_p.append(ns_shock_p_dict[date])
        
        dta_ns_shocks_r.append(ns_shock_r_dict[date])
    
    else:
        
        dta_s_shocks_p.append(np.nan)
        
        dta_s_shocks_r.append(np.nan)
        
        dta_ns_shocks_p.append(np.nan)
        
        dta_ns_shocks_r.append(np.nan)
    
    if date in cpi_growth_index:
        
        dta_cpi.append(cpi_dict[date])
        
    else:
        
        dta_cpi.append(np.nan)
    
    ob = ob + 1

dta_df['s_shock_p'] = dta_s_shocks_p

dta_df['s_shock_r'] = dta_s_shocks_r

dta_df['ns_shock_p'] = dta_ns_shocks_p

dta_df['ns_shock_r'] = dta_ns_shocks_r

dta_df['cpi_increase'] = dta_cpi

### Getting True Quarter Variable

This block gets the true year quarter to enable quarterly industry time dummies, where 1960q1 = 0. Since this takes the start month of the quarter from which an observation is taken, a convention must be established regarding what months correspond to which quarter. I define...

- The first quarter of a year to be observations from quarters starting in December (of the previous year), January and February
- The second quarter of a year to be observations from quarters starting in March, April and May
- The third quarter of a year to be observations from quarters starting in June, July and August
- The fourth quarter of a year to be observations from quarters starting in September, October and November

In [324]:
true_q = []

for date in dta_qstarts:
    
    q = int(((date.year - 1960)*12 + date.month)/3) # int() rounds a number down by default
    
    true_q.append(q)
    
dta_df['true_q'] = true_q

### Get Starting and Ending True Quarters

In [325]:
print(min(true_q),max(true_q))

130 223


### Export to *.dta*

In [326]:
dta_df.to_stata('Compustat_93Q1_14Q4_3.dta')